## Code template for using logical features for tweets emergency rescue requests identification
### Created by: Wael Khallouli
### Paper: intelligent agent...

-----------

## Imports

In [3]:
########################################################
############### Imports ################################
########################################################
from string import punctuation
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import pandas as pd
import numpy as np
import collections
from collections import Counter

from sklearn.model_selection import train_test_split
import string

import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))

### imports (1) ##
import pandas as pd
import numpy as pn
from numpy import mean

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

import string

import collections
from collections import Counter

### imports (2) ##
from string import punctuation
from os import listdir
from numpy import array

from pickle import load
from numpy import array


### imports (4) ##
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate


In [4]:
from sklearn.svm import SVC
from Baseline_Models import Display_metrics,Display_classification_report,Confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef

## Read Harvey data

In [27]:
########################################################
############### Read data ##############################
########################################################
file_path = '/home/wkhal001/Desktop/data_rescue_mining/labeled_ds_Corrected_csv.csv'

### Read data set 
labeledDF=pd.read_csv(file_path) 

#labeledDF.drop(['id','Unnamed: 0','loc','situ','save','sos','address','sos.pred'],1)
labeledDF = labeledDF.drop(['id','Unnamed: 0','loc','situ','save','sos','address','sos.pred'],1)

Training_set = labeledDF[['text','sos.correct']]

/home/wkhal001/envs/Twitter_crisis/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


In [28]:
Training_set

,text,sos.correct
0,#Harvey floods TV station #KHOU in #Houston. h...,0
1,"@RandiRhodes RR call him out for visiting SA, ...",0
2,Wow a tv station is flooding in Houston! So sc...,0
3,"My son, dil &amp; 2 grandkids in grand lakes, ...",0
4,is the beltway still flooded? ya boy need to g...,0
...,...,...
5787,"Around 10,000,000,000,000 gallons of water fro...",0
5788,The road to my residence is flooded. Thank God...,0
5789,Texas road closures and flooding kept up to da...,0
5790,"@HellerWeather Tim, any maps to show where flo...",0


In [29]:
Training_set_positives = Training_set[Training_set['sos.correct'] ==  1]

In [30]:
Training_set_positives

,text,sos.correct
30,Urgently need #WaterRescue at 10415 Merry Mead...,1
42,Folks looking for boat assistance near - Hardy...,1
52,@houstonpolice please help my sister !! 🙏🏻🙏🏻#h...,1
70,"Elderly Frank Emmitte trapped in 3226 Ave G, D...",1
86,Grant rd &amp; lake wood forest in houston. ne...,1
...,...,...
5166,@L0stSandal 777 Coolwood Dr Houston 8 Wk son n...,1
5167,.@houstonpolice help needed in 4055 South Brae...,1
5177,Please help my mother-in-law: a cancer patient...,1
5197,@HarveyRescue @HarveyRelief #HARVEY SOS: 2121 ...,1


In [31]:
Training_set['f1_1'] = 0
Training_set['f1_2'] = 0
Training_set['f2'] = 0
Training_set['f3_1'] = 0
Training_set['f3_2'] = 0
Training_set['f4'] = 0
Training_set['f5'] = 0
Training_set['f6'] = 0
Training_set['f7'] = 0
Training_set['f8'] = 0

/home/wkhal001/envs/Twitter_crisis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
Training_set.columns

Index(['text', 'sos.correct', 'f1_1', 'f1_2', 'f2', 'f3_1', 'f3_2', 'f4', 'f5',
       'f6', 'f7', 'f8'],
      dtype='object')

## Logical Approach: filer data set

In [33]:
import re

In [34]:
#--------------------  Create filter 1 ---------------------------------------#
# Feature 1.1 keywords with hurricane/flood
rgx_f1_1 = "\\b(Hurricane|HurricaneHarvey|Harvey2017|HARVEYHELP|HarveyStorm|harveyhouston|houstonflood|houstonfloods|houstonflooding|texasflood|texasfloods|texasflooding|harveyflood|harveyflooding|HurricaneFlood|HurricaneSOS)\\b"

# Feature 1.2 keywords with situation descriptions
rgx_f1_2 = "\\b(stranded|stuck|trapped|traps?|trapping|roofs?|rooftop|injured|hurt)\\b"

# Feature 1.3: contain both the following two keyword groups (ignore case):
# group 1: names of cities and towns near Houston
# group 2: flood related keywords. e.g. flood, flooding
rgx_f1_3a = "\\b(Houston|Texas|Galveston|Lake\\sJackson|Pasadena|League\\sCity|DICKINSON|Pearland|Missouri\\sCity|Sugar\\sLand|Richmond|Rosenberg|Alvin|Baytown|Fresno|Mont\\sBelvieu|Humble|Woodlands|Spring|Tomball|Cypress|Brookshire\\sKaty|FRIENDSWOOD)\\b"
rgx_f1_3b = "\\b(flood|floods|flooding|flooded)\\b"

#------------------ Create filter 2: Requesting rescue ------------------------#
rgx_f2 = "\\b(rescue|rescues|rescuing|rescued|helps?|helping|WaterRescue|WaterRescueNeeded|aid|assistance|boats?|HarveyRescue|HarveySOS|HurricaneRescue|FloodRescue|HurricaneSOS|HarveyRelief)\\b"

# ------ Create filter 3: address description-----------------------------------#
address_pattern = "(\\b\\d+\\s+(#?[A-z]+\\.?(-[A-z]+)?\\s+){1,3}\\b(#@#|ANNEX|ANEX|ANNX|ANX|#@#|ARCADE|ARC|#@#|AVENUE|AV|AVE|AVEN|AVENU|AVN|AVNUE|#@#|BAYOU|BAYOO|BYU|BAYOU|#@#|BEACH|BCH|BEACH|#@#|BEND|BND|#@#|BLUFF|BLF|BLUF|BLUFFS|BLFS|#@#|BOTTOM|BOT|BTM|BOTTM|#@#|BOULEVARD|BLVD|BOUL|BOULV|#@#|BRANCH|BR|BRNCH|#@#|BRIDGE|BRDGE|BRG|#@#|BROOK|BRK|BROOKS|BRKS|#@#|BURG|BG|BURGS|BGS|#@#|BYPASS|BYP|BYPA|BYPAS|BYPS|#@#|CAMP|CP|CMP|#@#|CANYON|CANYN|CYN|CNYN|#@#|CAPE|CPE|#@#|CAUSEWAY|CSWY|CAUSWA|CSWY|#@#|CENTER|CEN|CTR|CENT|CENTR|CENTRE|CNTER|CNTR|CENTERS|CTRS|#@#|CIRCLE|CIR|CIRC|CIRCL|CRCL|CRCLE|CIRCLES|CIRS|#@#|CLIFF|CLF|CLIFFS|CLFS|CLIFFS|#@#|CLUB|CLB|#@#|COMMON|CMN|COMMONS|CMNS|#@#|CORNER|COR|CORNERS|CORS|#@#|COURSE|CRSE|#@#|COURT|CT|COURTS|CTS|#@#|COVE|CV|COVES|COVES|CVS|#@#|CREEK|CRK|#@#|CRESCENT|CRES|CRSENT|CRSNT|CREST|CRST|CR|#@#|CROSSING|CRSSNG|XING|#@#|CROSSROAD|XRD|CROSSROADS|XRDS|#@#|CURVE|CURV|#@#|DALE|DL|#@#|#DAM#|#DM#|#@#|DIVIDE|DIV|DV|DIVIDE|DVD|#@#|DRIVE|DR|DRIV|DRV|DRIVES|DRS|#@#|ESTATE|EST|ESTATES|ESTS|#@#|EXPRESSWAY|EXP|EXPY|EXPR|EXPRESS|EXPW|#@#|EXTENSION|EXT|EXTN|EXTNSN|EXTENSIONS|EXTS|#@#|FALL|FALLS|FLS|#@#|FERRY|FRY|FRRY|FRY|#@#|FIELD|FLD|FIELDS|FLDS|#@#|FLAT|FLT|FLATS|FLTS|#@#|FORD|FRD|FORDS|FRDS|#@#|FOREST|FRST|FORESTS|FRST|#@#|FORGE|FORG|FRG|FORGES|FRGS|#@#|FORK|FRK|FORKS|FRKS|#@#|FORT|FT|FRT|#@#|FREEWAY|FWY|FREEWY|FRWAY|FRWY|FWY|#@#|GARDEN|GDN|GRDN|GARDENS|GDNS|GRDNS|#@#|GATEWAY|GTWY|GATEWY|GATWAY|GTWAY|GTWY|#@#|GLEN|GLN|GLENS|GLNS|#@#|GREEN|GRN|GREENS|GRNS|#@#|GROVE|GROV|GRV|GROVES|GRVS|#@#|HARBOR|HARB|HBR|HARBR|HRBOR|HARBORS|HBRS|#@#|HAVEN|HVN|#@#|HEIGHTS|HT|HTS|#@#|HIGHWAY|HWY|HIWAY|HIWY|HWAY|HWY|#@#|HILL|HL|HILLS|HLS|#@#|HOLLOW|HLLW|HOLW|HOLLOWS|HOLWS|#@#|INLET|INLT|#@#|ISLAND|#IS#|ISLND|ISLANDS|ISS|ISLNDS|#@#|ISLE|ISLES|#@#|JUNCTION|JCT|JCTION|JCTN|JUNCTN|JUNCTON|JUNCTIONS|JCTNS|JCTS|JUNCTIONS|#@#|KEY|KY|KEYS|KYS|#@#|KNOLL|KNL|KNOL|KNOLLS|KNLS|#@#|LAKE|LK|LAKES|LKS|#@#|LAND|LANDING|LNDG|LNDNG|#@#|LANE|LN|#@#|LIGHT|LGT|LIGHTS|LGTS|#@#|LOAF|LF|#@#|LOCK|LCK|LOCKS|LCKS|#@#|LODGE|LDG|LDGE|#@#|LOOP|LOOPS|#@#|MALL|#@#|MANOR|MNR|MANORS|MNRS|#@#|MEADOW|MDW|MEADOWS|MDWS|#@#|MEWS|#@#|MILL|ML|MILLS|MLS|#@#|MISSION|MISSN|MSN|MSSN|#@#|MOTORWAY|MTWY|#@#|MOUNT|MNT|MT|MOUNTAIN|MNTAIN|MTN|MNTN|MOUNTIN|MTIN|MOUNTAINS|MNTNS|MTNS|#@#|NECK|NCK|#@#|ORCHARD|ORCH|ORCHRD|#@#|OVAL|OVL|#@#|OVERPASS|OPAS|#@#|PARK|PRK|PARKS|#@#|PARKWAY|PARKWY|PKWAY|PKWY|PKY|PARKWAYS|PKWYS|#@#|PASS|PASSAGE|PSGE|#@#|#PATH#|#PATHS#|#@#|PIKE|PIKES|#@#|PINE|PNE|PINES|PNES|#@#|PLACE|PL|#@#|PLAIN|PLN|PLAINS|PLNS|#@#|PLAZA|PLZ|PLZA|#@#|#POINT#|PT|#POINTS#|PTS|#@#|PORT|PRT|PORTS|PRTS|#@#|PRAIRIE|PR|PRR|#@#|RADIAL|RAD|RADL|RADIEL|RADL|#@#|RAMP|#@#|RANCH|RNCH|RANCHES|RNCHS|#@#|RAPID|RPD|RAPIDS|RPDS|#@#|REST|RST|#@#|RIDGE|RDG|RDGE|RIDGES|RDGS|#@#|RIVER|RIV|RVR|RIVR|#@#|ROAD|RD|ROADS|RDS|#@#|ROUTE|RTE|#@#|ROW|#@#|RUE|#@#|#RUN#|#@#|SHOAL|SHL|SHL|SHOALS|SHLS|#@#|SHORE|SHOAR|SHR|SHORES|SHOARS|SHRS|#@#|SKYWAY|SKWY|#@#|SPRING|SPG|SPNG|SPRINGS|SPGS|SPNGS|#@#|SPUR|SPURS|#@#|SQUARE|SQ|SQR|SQRE|SQU|SQUARES|SQRS|#@#|STATION|STA|STATN|STN|#@#|STRAVENUE|STRA|STRAV|STRAVEN|STRAVN|STRVN|STRVNUE|#@#|STREAM|STRM|STREME|#@#|STREET|ST|STRT|STR|STREETS|STS|#@#|SUMMIT|SMT|SUMIT|SUMITT|SUMMIT|#@#|TERRACE|TER|TERR|#@#|THROUGHWAY|TRWY|#@#|TRACE|TRCE|TRACES|TRCE|#@#|TRACK|TRAK|TRACKS|TRAK|TRK|TRKS|#@#|TRAFFICWAY|TRFY|#@#|TRAIL|TRL|TRAILS|TRL|TRLS|#@#|TRAILER|TRLR|TRLRS|#@#|TUNNEL|TUNL|TUNLS|TUNNELS|TUNNL|#@#|TURNPIKE|TRNPK|TPKE|TURNPK|#@#|UNDERPASS|UPAS|#@#|UNION|UN|UNIONS|UNS|#@#|VALLEY|VLY|VALLY|VLLY|VLY|VALLEYS|VLYS|#@#|VIADUCT|VDCT|#VIA#|VIADCT|#@#|VIEW|VW|VIEWS|VWS|#@#|VILLAGE|VILL|VLG|VILLG|VILLIAGE|VILLAGES|VLGS|#@#|VILLE|VL|#@#|VISTA|VIS|VIST|VST|VSTA|#@#|#WALK#|#WALKS#|#@#|#WALL#|#@#|#WAY#|WY|#WAYS#|#@#|#WELL#|WL|WELLS|WLS|#@#)\\.?\\b|#@#)|(\\b\\d+\\s+(AVENUE|AV|AVE|AVEN|AVENU|AVN|AVNUE|#@#|HIGHWAY|HWY|HIWAY|HIWY|HWAY|#@#|ROAD|RD|ROADS|RDS|#@#|ROUTE|RTE|#@#|STREET|ST|STRT|STR|STREETS|STS|#@#)\\.?\\s+([A-z]|\\d+)\\b)"
non_address_pattern = "\\b\\d+\\s+(am|pm|hrs?|hours?|mins?|minutes?|seconds?|#@#|days?|months?|mon|yr|yrs|years?|#@#|ft|feet|foot|in|inch(es)?|meters?|miles?|#@#|pounds?|pnd|ounce|oz|kg|kilograms?|grams?|tons?|#@#|gallons?|liters?|cubes?|volumes?|quarts?|bottles?|cups?|#@#|per|per\\s+cent|percent|degrees?|times?|#@#|dollars?|USD|GBP|hundreds?|thousands?|millions?|billions?|trillions?|#@#|from|to|at|and|or|were|are|Fan\\sClub|live|hurricane|exit|entrance|#@#|rescued|donation|people|patients|seniors|elderly|women|children|clergy|#@#)\\.?\\s+(#?[A-z]+\\.?(-[A-z]+)?\\s+){0,2}\\b(#@#|ANNEX|ANEX|ANNX|ANX|#@#|ARCADE|ARC|#@#|AVENUE|AV|AVE|AVEN|AVENU|AVN|AVNUE|#@#|BAYOU|BAYOO|BYU|BAYOU|#@#|BEACH|BCH|BEACH|#@#|BEND|BND|#@#|BLUFF|BLF|BLUF|BLUFFS|BLFS|#@#|BOTTOM|BOT|BTM|BOTTM|#@#|BOULEVARD|BLVD|BOUL|BOULV|#@#|BRANCH|BR|BRNCH|#@#|BRIDGE|BRDGE|BRG|#@#|BROOK|BRK|BROOKS|BRKS|#@#|BURG|BG|BURGS|BGS|#@#|BYPASS|BYP|BYPA|BYPAS|BYPS|#@#|CAMP|CP|CMP|#@#|CANYON|CANYN|CYN|CNYN|#@#|CAPE|CPE|#@#|CAUSEWAY|CSWY|CAUSWA|CSWY|#@#|CENTER|CEN|CTR|CENT|CENTR|CENTRE|CNTER|CNTR|CENTERS|CTRS|#@#|CIRCLE|CIR|CIRC|CIRCL|CRCL|CRCLE|CIRCLES|CIRS|#@#|CLIFF|CLF|CLIFFS|CLFS|CLIFFS|#@#|CLUB|CLB|#@#|COMMON|CMN|COMMONS|CMNS|#@#|CORNER|COR|CORNERS|CORS|#@#|COURSE|CRSE|#@#|COURT|CT|COURTS|CTS|#@#|COVE|CV|COVES|COVES|CVS|#@#|CREEK|CRK|#@#|CRESCENT|CRES|CRSENT|CRSNT|CREST|CRST|CR|#@#|CROSSING|CRSSNG|XING|#@#|CROSSROAD|XRD|CROSSROADS|XRDS|#@#|CURVE|CURV|#@#|DALE|DL|#@#|#DAM#|DM|#@#|DIVIDE|DIV|DV|DIVIDE|DVD|#@#|DRIVE|DR|DRIV|DRV|DRIVES|DRS|#@#|ESTATE|EST|ESTATES|ESTS|#@#|EXPRESSWAY|EXP|EXPY|EXPR|EXPRESS|EXPW|#@#|EXTENSION|EXT|EXTN|EXTNSN|EXTENSIONS|EXTS|#@#|FALL|FALLS|FLS|#@#|FERRY|FRY|FRRY|FRY|#@#|FIELD|FLD|FIELDS|FLDS|#@#|FLAT|FLT|FLATS|FLTS|#@#|FORD|FRD|FORDS|FRDS|#@#|FOREST|FRST|FORESTS|FRST|#@#|FORGE|FORG|FRG|FORGES|FRGS|#@#|FORK|FRK|FORKS|FRKS|#@#|FORT|FT|FRT|#@#|FREEWAY|FWY|FREEWY|FRWAY|FRWY|FWY|#@#|GARDEN|GDN|GRDN|GARDENS|GDNS|GRDNS|#@#|GATEWAY|GTWY|GATEWY|GATWAY|GTWAY|GTWY|#@#|GLEN|GLN|GLENS|GLNS|#@#|GREEN|GRN|GREENS|GRNS|#@#|GROVE|GROV|GRV|GROVES|GRVS|#@#|HARBOR|HARB|HBR|HARBR|HRBOR|HARBORS|HBRS|#@#|HAVEN|HVN|#@#|HEIGHTS|HT|HTS|#@#|HIGHWAY|HWY|HIWAY|HIWY|HWAY|HWY|#@#|HILL|HL|HILLS|HLS|#@#|HOLLOW|HLLW|HOLW|HOLLOWS|HOLWS|#@#|INLET|INLT|#@#|ISLAND|#IS#|ISLND|ISLANDS|ISS|ISLNDS|#@#|ISLE|ISLES|#@#|JUNCTION|JCT|JCTION|JCTN|JUNCTN|JUNCTON|JUNCTIONS|JCTNS|JCTS|JUNCTIONS|#@#|KEY|KY|KEYS|KYS|#@#|KNOLL|KNL|KNOL|KNOLLS|KNLS|#@#|LAKE|LK|LAKES|LKS|#@#|LAND|LANDING|LNDG|LNDNG|#@#|LANE|LN|#@#|LIGHT|LGT|LIGHTS|LGTS|#@#|LOAF|LF|#@#|LOCK|LCK|LOCKS|LCKS|#@#|LODGE|LDG|LDGE|#@#|LOOP|LOOPS|#@#|MALL|#@#|MANOR|MNR|MANORS|MNRS|#@#|MEADOW|MDW|MEADOWS|MDWS|#@#|MEWS|#@#|MILL|ML|MILLS|MLS|#@#|MISSION|MISSN|MSN|MSSN|#@#|MOTORWAY|MTWY|#@#|MOUNT|MNT|MT|MOUNTAIN|MNTAIN|MTN|MNTN|MOUNTIN|MTIN|MOUNTAINS|MNTNS|MTNS|#@#|NECK|NCK|#@#|ORCHARD|ORCH|ORCHRD|#@#|OVAL|OVL|#@#|OVERPASS|OPAS|#@#|PARK|PRK|PARKS|#@#|PARKWAY|PARKWY|PKWAY|PKWY|PKY|PARKWAYS|PKWYS|#@#|PASS|PASSAGE|PSGE|#@#|#PATH#|#PATHS#|#@#|PIKE|PIKES|#@#|PINE|PNE|PINES|PNES|#@#|PLACE|PL|#@#|PLAIN|PLN|PLAINS|PLNS|#@#|PLAZA|PLZ|PLZA|#@#|#POINT#|PT|#POINTS#|PTS|#@#|PORT|PRT|PORTS|PRTS|#@#|PRAIRIE|PR|PRR|#@#|RADIAL|RAD|RADL|RADIEL|RADL|#@#|RAMP|#@#|RANCH|RNCH|RANCHES|RNCHS|#@#|RAPID|RPD|RAPIDS|RPDS|#@#|REST|RST|#@#|RIDGE|RDG|RDGE|RIDGES|RDGS|#@#|RIVER|RIV|RVR|RIVR|#@#|ROAD|RD|ROADS|RDS|#@#|ROUTE|RTE|#@#|ROW|#@#|RUE|#@#|#RUN#|#@#|SHOAL|SHL|SHL|SHOALS|SHLS|#@#|SHORE|SHOAR|SHR|SHORES|SHOARS|SHRS|#@#|SKYWAY|SKWY|#@#|SPRING|SPG|SPNG|SPRINGS|SPGS|SPNGS|#@#|SPUR|SPURS|#@#|SQUARE|SQ|SQR|SQRE|SQU|SQUARES|SQRS|#@#|STATION|STA|STATN|STN|#@#|STRAVENUE|STRA|STRAV|STRAVEN|STRAVN|STRVN|STRVNUE|#@#|STREAM|STRM|STREME|#@#|STREET|ST|STRT|STR|STREETS|STS|#@#|SUMMIT|SMT|SUMIT|SUMITT|SUMMIT|#@#|TERRACE|TER|TERR|#@#|THROUGHWAY|TRWY|#@#|TRACE|TRCE|TRACES|TRCE|#@#|TRACK|TRAK|TRACKS|TRAK|TRK|TRKS|#@#|TRAFFICWAY|TRFY|#@#|TRAIL|TRL|TRAILS|TRL|TRLS|#@#|TRAILER|TRLR|TRLRS|#@#|TUNNEL|TUNL|TUNLS|TUNNELS|TUNNL|#@#|TURNPIKE|TRNPK|TPKE|TURNPK|#@#|UNDERPASS|UPAS|#@#|UNION|UN|UNIONS|UNS|#@#|VALLEY|VLY|VALLY|VLLY|VLY|VALLEYS|VLYS|#@#|VIADUCT|VDCT|#VIA#|VIADCT|#@#|VIEW|VW|VIEWS|VWS|#@#|VILLAGE|VILL|VLG|VILLG|VILLIAGE|VILLAGES|VLGS|#@#|VILLE|VL|#@#|VISTA|VIS|VIST|VST|VSTA|#@#|#WALK#|#WALKS#|#@#|#WALL#|#@#|#WAY#|WY|#WAYS#|#@#|#WELL#|WL|WELLS|WLS|#@#)\\.?\\b"

# ------ Create filter 4: with key words of tweets about political -------#
rgx_f4 = "\\b(realDonaldTrump|Trump|DonaldTrump|BarackObama|Obama|Election|Election2016|vote|republicans|republican|democrats|democrat|GOP|dems|immigrant|immigrants|climate\\s?change|gas\\s?prices|ICE|buzzfeed(news)?|tedcruz|SenTedCruz)\\b"
# all tweets about ICE and buzzfeed are polical ones

# ------ Create filter 5: with key words of tweets about offering helps -------#
rgx_f5 = "\\b(donate|donating|donated|donations|Charity|Charities|church|shelters?|sheltering|(we|I)\\s+can\\s+help|open\\s+for\\s+helps?|drop\\s+off|HELP\\sthe\\s#?AmericanRedCross)\\b"

# ------ Create filter 6: with key words of tweets about commercial -------#
rgx_f6 = "\\$\\d+(.\\d{0,2})?|\\$\\$+|\\b(sales|for\\ssale|dollors|hundreds?|thousands?|millions?|billions?|trillions?|[A-z]*market)\\b"
# 2 regex, one  for market; one for open;


# ------ Create filter 7: with key words of tweets of newsreport -------#
rgx_f7 = "#BREAKING:\\s+|\\b(Press\\sConference|Live\\svideo\\sfeed|Live\\sStream|County\\sUpdate:|National\\s+Hurricane\\s+Center|Tropical\\s+Storm\\s+Harvey|MANDATORY\\s+EVACUATION|After\\s+Hurricane\\s+Harvey|Ahead\\s+of\\s+Hurricane\\s+Harvey|like\\s+a\\s+river|6\\s+mil\\s+people|High\\s+call\\s+volume|Epic\\s+flooding|cameras?|webcam|\\bFM\\s+\\d+|News\\s+in\\s+the\\s+#?DMV)\\b"
# SOS tweet not likely use "Tropical Storm Harvey" or "National Hurricane Center"
# typically used by news


# ------ Create filter 8: rescue status update --------------#
rgx_f8 = "-\\sAwaiting\\sUpdates?\\b|-\\sRescued!\\s|\\b(Ha(ve|s)\\sBeen\\sRescued)\\b"


In [35]:
### filter 1 final ####################
Training_set['f1_1'] = Training_set.apply(lambda row: 1 if re.findall(rgx_f1_1,row['text'],re.IGNORECASE) else 0,axis=1)
Training_set['f1_2'] = Training_set.apply(lambda row: 1 if re.findall(rgx_f1_2,row['text'],re.IGNORECASE) else 0,axis=1)

In [36]:
Training_set['f2'] = Training_set.apply(lambda row: 1 if re.findall(rgx_f2,row['text'],re.IGNORECASE) else 0,axis=1)

In [37]:
Training_set['f3_1'] = Training_set.apply(lambda row: 1 if re.findall(address_pattern,row['text'],re.IGNORECASE) else 0,axis=1)
Training_set['f3_2'] = Training_set.apply(lambda row: 1 if re.findall(non_address_pattern,row['text'],re.IGNORECASE) else 0,axis=1)

In [38]:
Training_set['f4'] = Training_set.apply(lambda row: 1 if re.findall(rgx_f4,row['text'],re.IGNORECASE) else 0,axis=1)

In [39]:
Training_set['f5'] = Training_set.apply(lambda row: 1 if re.findall(rgx_f5,row['text'],re.IGNORECASE) else 0,axis=1)

In [40]:
Training_set['f6'] = Training_set.apply(lambda row: 1 if re.findall(rgx_f6,row['text'],re.IGNORECASE) else 0,axis=1)

In [41]:
Training_set['f7'] = Training_set.apply(lambda row: 1 if re.findall(rgx_f7,row['text'],re.IGNORECASE) else 0,axis=1)

In [42]:
Training_set['f8'] = Training_set.apply(lambda row: 1 if re.findall(rgx_f8,row['text'],re.IGNORECASE) else 0,axis=1)

In [43]:
Training_set

,text,sos.correct,f1_1,f1_2,f2,f3_1,f3_2,f4,f5,f6,f7,f8
0,#Harvey floods TV station #KHOU in #Houston. h...,0,0,0,0,0,0,0,0,0,0,0
1,"@RandiRhodes RR call him out for visiting SA, ...",0,0,0,0,0,0,0,1,0,0,0
2,Wow a tv station is flooding in Houston! So sc...,0,0,0,0,0,0,0,0,0,0,0
3,"My son, dil &amp; 2 grandkids in grand lakes, ...",0,0,0,1,1,0,0,0,0,0,0
4,is the beltway still flooded? ya boy need to g...,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5787,"Around 10,000,000,000,000 gallons of water fro...",0,0,0,0,0,0,0,0,0,0,0
5788,The road to my residence is flooded. Thank God...,0,1,0,0,0,0,0,0,0,0,0
5789,Texas road closures and flooding kept up to da...,0,0,0,0,0,0,0,0,0,0,0
5790,"@HellerWeather Tim, any maps to show where flo...",0,0,0,0,0,0,0,0,0,0,0


## Clasification: evaluate the logical expression for each tweet

In [57]:
Training_set['filter_sos'] = 0

In [58]:
def logical_evaluation_address(x,y):
    if x == 1:
        if y == 0:
            return 1
        else:
            return 0
    else:
        return 0
    
def logical_evaluation_final(x,y):
    if x == 1:
        if y == 0:
            return 1
        else:
            return 0
    else:
        return 0

def evaluate_expression(f1_1,f1_2,f2,f3_1,f3_2,f4,f5,f6,f7,f8): 
    f1 = f1_1 or f1_2
    f3 = logical_evaluation_address(f3_1,f3_2)
    f1_union_f2= f1 or f2
    f1_f2_intersect_f3 = f1_union_f2 and f3
    
    excluded_filters = f4 or f5 or f6 or f7 or f8
    
    return logical_evaluation_final(f1_f2_intersect_f3,excluded_filters)

In [59]:
## run classifier ###
Training_set['filter_sos'] = Training_set.apply(lambda row: 1 if evaluate_expression(row['f1_1'],row['f1_2'],row['f2'],row['f3_1'],row['f3_2'],row['f4'],row['f5'],row['f6'],row['f7'],row['f8']) else 0,axis=1)

labels = Training_set['sos.correct']
preds = Training_set['filter_sos']


In [60]:
training_set_rescue = Training_set[Training_set['sos.correct'] == 1]

In [61]:
training_set_rescue

,text,sos.correct,f1_1,f1_2,f2,f3_1,f3_2,f4,f5,f6,f7,f8,filter_sos
30,Urgently need #WaterRescue at 10415 Merry Mead...,1,1,0,1,1,0,0,0,0,0,0,1
42,Folks looking for boat assistance near - Hardy...,1,1,0,1,0,0,0,0,0,0,0,0
52,@houstonpolice please help my sister !! 🙏🏻🙏🏻#h...,1,1,0,1,1,0,0,0,0,0,0,1
70,"Elderly Frank Emmitte trapped in 3226 Ave G, D...",1,1,1,1,1,0,0,0,0,0,0,1
86,Grant rd &amp; lake wood forest in houston. ne...,1,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5166,@L0stSandal 777 Coolwood Dr Houston 8 Wk son n...,1,1,0,1,1,0,0,0,0,0,0,1
5167,.@houstonpolice help needed in 4055 South Brae...,1,0,1,1,1,0,0,0,0,0,0,1
5177,Please help my mother-in-law: a cancer patient...,1,0,0,1,1,0,0,0,0,0,0,1
5197,@HarveyRescue @HarveyRelief #HARVEY SOS: 2121 ...,1,0,0,1,1,0,0,0,0,0,0,1


In [62]:
training_set_rescue.to_csv('/home/wkhal001/Desktop/filtered_logical_positives.csv')

## Evaluation of the approach

In [63]:
from sklearn.metrics import classification_report

In [64]:
dict_r = classification_report(labels, preds,output_dict = True)

dict_r

{'0': {'precision': 0.9954586739327884,
  'recall': 0.9893482578082686,
  'f1-score': 0.9923940601231439,
  'support': 5539},
 '1': {'precision': 0.794425087108014,
  'recall': 0.9011857707509882,
  'f1-score': 0.8444444444444444,
  'support': 253},
 'accuracy': 0.9854972375690608,
 'macro avg': {'precision': 0.8949418805204012,
  'recall': 0.9452670142796284,
  'f1-score': 0.9184192522837942,
  'support': 5792},
 'weighted avg': {'precision': 0.9866773380442062,
  'recall': 0.9854972375690608,
  'f1-score': 0.9859314819520958,
  'support': 5792}}

In [ ]:
##################################################
############# contextual words ###################
##################################################

# please help my sister
# Urgently need
# @houstonpolice
# Needs help
# trapped in
# IN NEED OF RESCUE
# Rescue for
# need rescuing
# Please send help
# please help my friend
# Please send help asap
# we are stuck on
# please share
# still waiting for
# PLZ HELP CALL 911
# Please RT
# Men stranded on
# life in danger
# RT emergency service
# they are trapped inside
# Pls send help 
# they need rescue help 
# have been trapped for
# Please retweet

#list_help_words_bigrams =[('please','help'),('Urgently','need'),('Needs','help'),('send','help'),('need','rescuing'),('PLZ','HELP')]
#list_retweet_help_bigrams = [('please','RT'),('please','share'),('please','retweet')]#
#list_situation_bigrams =[('stranded','in'),('trapped','inside')]

In [ ]:
##################################################
############# hashtags ###########################
##################################################
#HurricaneHarvey
#houstonflood
#Harveyflood
#hurricaneharvey
#WaterRescueNeeded
#houston #harvey #HARVEYHELP #HarveyRelief #HarveyStorm
#rescue #flood #Khou #harvey #help #retweet
#Help 
#HarveyFlood #HarveyRelief #Relief #harveyhouston #Houston
#harveysos
#houston
#harveyrescue #harveysos #harveystorm #Texas
#sendhelp  #Urgent #Houstonfloods 
#Harveyflood
#HoustonFloods #houstonPolice

